In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('substance p', 3292),
 ('side population', 691),
 ('single positive', 240),
 ('signal peptide', 135),
 ('seminal plasma', 118),
 ('sulfadoxine pyrimethamine', 109),
 ('sulphadoxine pyrimethamine', 95),
 ('sex peptide', 62),
 ('streptococcus pneumoniae', 54),
 ('surfactant protein', 48),
 ('sp600125', 47),
 ('serine protease', 45),
 ('senile plaques', 35),
 ('summating potential', 31),
 ('silent period', 30),
 ('spironolactone', 29),
 ('spirulina platensis', 24),
 ('surface plasmon', 24),
 ('short photoperiod', 23),
 ('spleen', 21),
 ('secondary progressive', 19),
 ('spontaneous proliferation', 18),
 ('social phobia', 18),
 ('soy protein', 18),
 ('spore photoproduct', 17),
 ('sulfapyridine', 17),
 ('specific', 16),
 ('subplate', 14),
 ('serine proline', 13),
 ('spiropyran', 13),
 ('silk peptide', 13),
 ('sulphate + phosphate', 13),
 ('sodium pyruvate', 12),
 ('synaptopodin', 12),
 ('stationary phase', 11),
 ('social participation', 11),
 ('sphingosine', 11),
 ('substrate protein', 11)

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'CHEBI:CHEBI:80308': 'Substance P',
 'MESH:D010949': 'Plasma',
 'MESH:C001205': 'fanasil, pyrimethamine drug combination',
 'CHEBI:CHEBI:16670': 'peptide',
 'MESH:D013296': 'Streptococcus pneumoniae',
 'CHEBI:CHEBI:90695': 'anthra[1,9-cd]pyrazol-6(2H)-one',
 'CHEBI:CHEBI:9241': 'spironolactone',
 'MESH:C000651671': 'Arthrospira platensis',
 'MESH:D017440': 'Photoperiod',
 'MESH:D013154': 'Spleen',
 'MESH:D000072861': 'Phobia, Social',
 'CHEBI:CHEBI:36080': 'protein',
 'CHEBI:CHEBI:139518': '(5R)-5,6-dihydro-5-(thymidin-7-yl)thymidine dimer(2-) residue',
 'CHEBI:CHEBI:132842': 'sulfapyridine',
 'CHEBI:CHEBI:26271': 'proline',
 'MESH:C088184': 'spiropyran',
 'CHEBI:CHEBI:50144': 'sodium pyruvate',
 'HGNC:30672': 'SYNPO',
 'GO:GO:0070314': 'G1 to G0 transition',
 'MESH:D058992': 'Social Participation',
 'CHEBI:CHEBI:16393': 'sphingosine',
 'CHEBI:CHEBI:17203': 'L-proline',
 'MESH:D011698': 'Pursuit, Smooth',
 'CHEBI:CHEBI:132106': 'sodium propionate',
 'CHEBI:CHEBI:34922': 'picloram',
 '

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-04 03:12:53] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'s 1 plus cisplatin': 'CHEBI:CHEBI:27899',
  's haematobium positive': 'ungrounded',
  's pneumoniae': 'ungrounded',
  's pyogenes': 'ungrounded',
  'salmonella pullorum': 'ungrounded',
  'salvadora persica': 'ungrounded',
  'salvadora persica l': 'ungrounded',
  'saponin': 'CHEBI:CHEBI:26605',
  'sapota powder': 'ungrounded',
  'sarcoplasmic protein': 'ungrounded',
  'saturating pulse': 'ungrounded',
  'scaffolding protein': 'ungrounded',
  'schizolobium parahyba': 'ungrounded',
  'schizolobium parahyba aqueous extract': 'ungrounded',
  'schizophrenia': 'MESH:D012559',
  'schizophrenia patients': 'MESH:D012559',
  'sclerosing pneumocytoma': 'MESH:D047868',
  'seasonal pattern': 'ungrounded',
  'secondary pharmacophore': 'ungrounded',
  'secondary preventive': 'ungrounded',
  'secondary progressive': 'ungrounded',
  'secretion production': 'ungrounded',
  'secretion protein': 'ungrounded',
  'selection positive': 'ungrounded',
  'selenium enriched probiotics': 'MESH:D019936',
  'self

In [25]:
grounding_map, names, pos_labels = [{'s 1 plus cisplatin': 'CHEBI:CHEBI:27899',
  's haematobium positive': 'ungrounded',
  's pneumoniae': 'TAXONOMY:1313',
  's pyogenes': 'ungrounded',
  'salmonella pullorum': 'ungrounded',
  'salvadora persica': 'ungrounded',
  'salvadora persica l': 'ungrounded',
  'saponin': 'CHEBI:CHEBI:26605',
  'sapota powder': 'ungrounded',
  'sarcoplasmic protein': 'ungrounded',
  'saturating pulse': 'ungrounded',
  'scaffolding protein': 'ungrounded',
  'schizolobium parahyba': 'ungrounded',
  'schizolobium parahyba aqueous extract': 'ungrounded',
  'schizophrenia': 'MESH:D012559',
  'schizophrenia patients': 'MESH:D012559',
  'sclerosing pneumocytoma': 'MESH:D047868',
  'seasonal pattern': 'ungrounded',
  'secondary pharmacophore': 'ungrounded',
  'secondary preventive': 'ungrounded',
  'secondary progressive': 'ungrounded',
  'secretion production': 'ungrounded',
  'secretion protein': 'ungrounded',
  'selection positive': 'ungrounded',
  'selenium enriched probiotics': 'MESH:D019936',
  'self pruning': 'ungrounded',
  'seminal plasma': 'ungrounded',
  'senile plaques': 'ungrounded',
  'separated pair': 'ungrounded',
  'sepiapterin': 'CHEBI:CHEBI:16095',
  'septal positive': 'ungrounded',
  'ser pro rich': 'ungrounded',
  'serine proline': 'ungrounded',
  'serine proline rich': 'ungrounded',
  'serine protease': 'HGNC:9475',
  'serine protease domain': 'HGNC:9475',
  'serine proteinase': 'HGNC:9475',
  'seroprotection': 'ungrounded',
  'serum phosphorus': 'CHEBI:CHEBI:28659',
  'sevoflurane postconditioning': 'ungrounded',
  'sex peptide': 'ungrounded',
  'short days photoperiod': 'ungrounded',
  'short peptide': 'ungrounded',
  'short photoperiod': 'ungrounded',
  'shoulder pain': 'MESH:D020069',
  'sickness presence': 'ungrounded',
  'sickness presenteeism': 'ungrounded',
  'side population': 'MESH:D058985',
  'side population cells': 'MESH:D058985',
  'siegesbeckia pubescens makino': 'ungrounded',
  'signal peptide': 'ungrounded',
  'silent period': 'ungrounded',
  'silk peptide': 'ungrounded',
  'simple phoneme': 'ungrounded',
  'simultaneously possible': 'ungrounded',
  'single particle band gap': 'ungrounded',
  'single perovskite': 'MESH:C059910',
  'single port': 'ungrounded',
  'single positive': 'ungrounded',
  'single positive thymocytes': 'ungrounded',
  'sirna pool': 'ungrounded',
  'site preference': 'ungrounded',
  'sleep paralysis': 'MESH:D020188',
  'slow pathway': 'CHEBI:CHEBI:34922',
  'slow potential': 'ungrounded',
  'slow progressors': 'ungrounded',
  'smart pass': 'ungrounded',
  'smooth pursuit': 'MESH:D011698',
  'social participation': 'MESH:D058992',
  'social performed': 'ungrounded',
  'social phobia': 'MESH:D000072861',
  'social preference': 'ungrounded',
  'sodium palmitate': 'CHEBI:CHEBI:15756',
  'sodium pentobarbital': 'CHEBI:CHEBI:7983',
  'sodium propionate': 'CHEBI:CHEBI:132106',
  'sodium pyruvate': 'ungrounded',
  'soluble p': 'ungrounded',
  'soluble protein': 'ungrounded',
  'sonication probe': 'ungrounded',
  'sonication probe method': 'ungrounded',
  'soy protein': 'ungrounded',
  'soybean protein': 'ungrounded',
  'sp': 'ungrounded',
  'sp treated rats': 'ungrounded',
  'sp waveform midpoint': 'ungrounded',
  'sp600125': 'CHEBI:CHEBI:90695',
  'spantide ii': 'MESH:C065446',
  'spasmolytic polypeptide': 'MESH:C451060',
  'spastic paresis': 'ungrounded',
  'specific': 'ungrounded',
  'specific phobia': 'DOID:DOID:599',
  'specific protein': 'ungrounded',
  'spermine': 'CHEBI:CHEBI:15746',
  'sphere': 'ungrounded',
  'sphinganine': 'CHEBI:CHEBI:16566',
  'sphingolipids': 'CHEBI:CHEBI:26739',
  'sphingosine': 'CHEBI:CHEBI:16393',
  'spike probability': 'ungrounded',
  'spike protein': 'ungrounded',
  'spin polar': 'ungrounded',
  'spions': 'CHEBI:CHEBI:139547',
  'spiramycin': 'CHEBI:CHEBI:85260',
  'spiro': 'ungrounded',
  'spironolactone': 'CHEBI:CHEBI:9241',
  'spiropyran': 'MESH:C088184',
  'spirulina': 'MESH:D053545',
  'spirulina platensis': 'ungrounded',
  'spirulina platensis water extract': 'ungrounded',
  'spleen': 'ungrounded',
  'splenectomy': 'MESH:D013156',
  'splenocytes': 'ungrounded',
  'splitomicin': 'CHEBI:CHEBI:75272',
  'spontaneous pneumothorax': 'ungrounded',
  'spontaneous proliferation': 'ungrounded',
  'spore density': 'ungrounded',
  'spore photoproduct': 'CHEBI:CHEBI:139518',
  'spray pyrolysis': 'MESH:D000077319',
  'stable peptide': 'ungrounded',
  'stance p': 'HGNC:11517',
  'standard patients': 'ungrounded',
  'standard precision': 'ungrounded',
  'standardised packaging': 'ungrounded',
  'stationary phase': 'GO:GO:0070314',
  'staurosporine': 'CHEBI:CHEBI:15738',
  'sterile pericarditis': 'ungrounded',
  'stratum pyramidale': 'ungrounded',
  'streptococcus pneumoniae': 'TAXONOMY:1313',
  'structure protein': 'ungrounded',
  'subplate': 'ungrounded',
  'substance p': 'HGNC:11517',
  'substrate protein': 'ungrounded',
  'succinoyl pyridine': 'ungrounded',
  'succinoylpyridine': 'ungrounded',
  'succinyl phosphonate': 'MESH:C099145',
  'sucrose phosphorylase': 'IP:IPR022527',
  'sucrose preference': 'ungrounded',
  'sugar phosphate': 'ungrounded',
  'sulfadoxine pyrimethamine': 'MESH:C001205',
  'sulfaphenazole': 'CHEBI:CHEBI:77780',
  'sulfapyridine': 'ungrounded',
  'sulfated polysaccharide': 'ungrounded',
  'sulfur polymer': 'CHEBI:CHEBI:60027',
  'sulphadoxine pyrimethamine': 'MESH:C001205',
  'sulphapyridine': 'CHEBI:CHEBI:132842',
  'sulphate + phosphate': 'ungrounded',
  'sulphate polysaccharide': 'ungrounded',
  'summating potential': 'ungrounded',
  'superficial parotidectomy': 'ungrounded',
  'superficial peroneal': 'ungrounded',
  'superficial pyramidal': 'ungrounded',
  'superplasticizer': 'ungrounded',
  'supine': 'ungrounded',
  'supine positive': 'ungrounded',
  'surepath': 'ungrounded',
  'surface plasmon': 'ungrounded',
  'surface potential': 'ungrounded',
  'surface pressure': 'ungrounded',
  'surfactant associated protein': 'ungrounded',
  'surfactant protein': 'HGNC:10798',
  'sweet potato': 'ungrounded',
  'synaptopodin': 'HGNC:30672',
  'synthetic peptide': 'ungrounded',
  'systolic blood pressure': 'EFO:0006335',
  'systolic pressure': 'EFO:0006335'},
 {'CHEBI:CHEBI:27899': 'cisplatin',
  'CHEBI:CHEBI:26605': 'saponin',
  'MESH:D012559': 'Schizophrenia',
  'MESH:D047868': 'Pulmonary Sclerosing Hemangioma',
  'MESH:D019936': 'Probiotics',
  'CHEBI:CHEBI:16095': 'sepiapterin',
  'HGNC:9475': 'PRSS1',
  'CHEBI:CHEBI:28659': 'phosphorus atom',
  'MESH:D020069': 'Shoulder Pain',
  'MESH:D058985': 'Side-Population Cells',
  'MESH:C059910': 'perovskite',
  'MESH:D020188': 'Sleep Paralysis',
  'CHEBI:CHEBI:34922': 'picloram',
  'MESH:D011698': 'Pursuit, Smooth',
  'MESH:D058992': 'Social Participation',
  'MESH:D000072861': 'Phobia, Social',
  'CHEBI:CHEBI:15756': 'hexadecanoic acid',
  'CHEBI:CHEBI:7983': 'pentobarbital',
  'CHEBI:CHEBI:132106': 'sodium propionate',
  'CHEBI:CHEBI:90695': 'anthra[1,9-cd]pyrazol-6(2H)-one',
  'MESH:C065446': 'spantide II',
  'MESH:C451060': 'spasmolytic polypeptide',
  'DOID:DOID:599': 'specific phobia',
  'CHEBI:CHEBI:15746': 'spermine',
  'CHEBI:CHEBI:16566': 'sphinganine',
  'CHEBI:CHEBI:26739': 'sphingolipid',
  'CHEBI:CHEBI:16393': 'sphingosine',
  'CHEBI:CHEBI:139547': 'superparamagnetic iron oxide nanoparticle',
  'CHEBI:CHEBI:85260': 'spiramycin I',
  'CHEBI:CHEBI:9241': 'spironolactone',
  'MESH:C088184': 'spiropyran',
  'MESH:D053545': 'Spirulina',
  'MESH:D013156': 'Splenectomy',
  'CHEBI:CHEBI:75272': 'splitomicin',
  'CHEBI:CHEBI:139518': '(5R)-5,6-dihydro-5-(thymidin-7-yl)thymidine dimer(2-) residue',
  'MESH:D000077319': 'Pyrolysis',
  'HGNC:11517': 'TAC1',
  'GO:GO:0070314': 'G1 to G0 transition',
  'CHEBI:CHEBI:15738': 'staurosporine',
  'TAXONOMY:1313': 'Streptococcus pneumoniae',
  'MESH:C099145': 'succinyl phosphonate',
  'IP:IPR022527': 'Sucrose phosphorylase',
  'MESH:C001205': 'fanasil, pyrimethamine drug combination',
  'CHEBI:CHEBI:77780': 'sulfaphenazole',
  'CHEBI:CHEBI:60027': 'polymer',
  'CHEBI:CHEBI:132842': 'sulfapyridine',
  'HGNC:10798': 'SFTPA1',
  'HGNC:30672': 'SYNPO',
  'EFO:0006335': 'systolic blood pressure'},
 ['CHEBI:CHEBI:90695', 'HGNC:10798', 'HGNC:11517', 'HGNC:9475']]

In [26]:
excluded_longforms = ['sp', 'sp treated rats', 'sp waveform midpoint']

In [27]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [28]:
additional_entities = {}

In [29]:
unambiguous_agent_texts = {}

In [30]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [18]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [19]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [31]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-04 03:44:02] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-04 03:47:40] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9529759743141945 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [32]:
classifier.stats

{'label_distribution': {'ungrounded': 893,
  'HGNC:11517': 2649,
  'CHEBI:CHEBI:34922': 9,
  'CHEBI:CHEBI:15738': 8,
  'CHEBI:CHEBI:15746': 3,
  'CHEBI:CHEBI:9241': 26,
  'CHEBI:CHEBI:7983': 4,
  'EFO:0006335': 9,
  'CHEBI:CHEBI:132842': 5,
  'HGNC:9475': 49,
  'CHEBI:CHEBI:28659': 5,
  'CHEBI:CHEBI:77780': 4,
  'CHEBI:CHEBI:16393': 8,
  'CHEBI:CHEBI:16566': 4,
  'MESH:C451060': 2,
  'HGNC:10798': 35,
  'MESH:D011698': 7,
  'CHEBI:CHEBI:139518': 14,
  'GO:GO:0070314': 9,
  'MESH:C001205': 124,
  'MESH:D058985': 398,
  'TAXONOMY:1313': 29,
  'MESH:C099145': 7,
  'MESH:D000072861': 12,
  'CHEBI:CHEBI:85260': 2,
  'CHEBI:CHEBI:90695': 30,
  'MESH:D020188': 5,
  'HGNC:30672': 8,
  'CHEBI:CHEBI:16095': 4,
  'CHEBI:CHEBI:132106': 9,
  'MESH:C088184': 12,
  'MESH:D012559': 6,
  'MESH:C065446': 3,
  'CHEBI:CHEBI:26739': 3,
  'CHEBI:CHEBI:27899': 2,
  'IP:IPR022527': 2,
  'MESH:D053545': 3,
  'MESH:D013156': 2,
  'MESH:D020069': 2,
  'MESH:D019936': 5,
  'MESH:C059910': 1,
  'CHEBI:CHEBI:75272'

In [33]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [34]:
disamb.dump(model_name, results_path)

In [35]:
print(disamb.info())

Disambiguation model for SP

Produces the disambiguations:
	(5R)-5,6-dihydro-5-(thymidin-7-yl)thymidine dimer(2-) residue	CHEBI:CHEBI:139518
	G1 to G0 transition	GO:GO:0070314
	PRSS1*	HGNC:9475
	Phobia, Social	MESH:D000072861
	Probiotics	MESH:D019936
	Pulmonary Sclerosing Hemangioma	MESH:D047868
	Pursuit, Smooth	MESH:D011698
	Pyrolysis	MESH:D000077319
	SFTPA1*	HGNC:10798
	SYNPO	HGNC:30672
	Schizophrenia	MESH:D012559
	Shoulder Pain	MESH:D020069
	Side-Population Cells	MESH:D058985
	Sleep Paralysis	MESH:D020188
	Social Participation	MESH:D058992
	Spirulina	MESH:D053545
	Splenectomy	MESH:D013156
	Streptococcus pneumoniae	TAXONOMY:1313
	Sucrose phosphorylase	IP:IPR022527
	TAC1*	HGNC:11517
	anthra[1,9-cd]pyrazol-6(2H)-one*	CHEBI:CHEBI:90695
	cisplatin	CHEBI:CHEBI:27899
	fanasil, pyrimethamine drug combination	MESH:C001205
	hexadecanoic acid	CHEBI:CHEBI:15756
	pentobarbital	CHEBI:CHEBI:7983
	perovskite	MESH:C059910
	phosphorus atom	CHEBI:CHEBI:28659
	picloram	CHEBI:CHEBI:34922
	polymer	CHEBI:

In [36]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1